# Text Classification with Magnitude & word2vec

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard
from pymagnitude import Magnitude
from keras.utils import Sequence

print(tf.__version__)

1.13.0-rc2


In [2]:
# Download the data from GCS
!wget 'https://storage.googleapis.com/movies_data/movies_metadata.csv'

--2019-02-27 17:15:46--  https://storage.googleapis.com/movies_data/movies_metadata.csv


Resolving storage.googleapis.com... 172.217.26.48
Connecting to storage.googleapis.com|172.217.26.48|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 34445126 (33M) [application/octet-stream]


Saving to: ‘movies_metadata.csv.10’

movies_metadata.csv   0%[                    ]       0  --.-KB/s               

...

movies_metadata.csv  97%[==================> ]  31.99M   814KB/s    eta 1s     

movies_metadata.csv  98%[==================> ]  32.24M   827KB/s    eta 1s     

movies_metadata.csv  99%[==================> ]  32.60M   881KB/s    eta 1s     

movies_metadata.csv  99%[==================> ]  32.82M   892KB/s    eta 1s     

movies_metadata.csv 100%[===================>]  32.85M   896KB/s    in 34s     

2019-02-27 17:16:21 (977 KB/s) - ‘movies_metadata.csv.10’ saved [34445126/34445126]



In [3]:
data = pd.read_csv('movies_metadata.csv')
data.head()

/Users/smap3/.local/share/virtualenvs/tensorflow_hub_vs_magnitude-pY8tZzeD/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
# urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/descriptions.p', 'descriptions.p')
# urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/genres.p', 'genres.p')

descriptions = pickle.load(open('pickle/descriptions.p', 'rb'))
genres = pickle.load(open('pickle/genres.p', 'rb'))



In [5]:
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = genres[train_size:]

In [7]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first movie in our training dataset
print(encoder.classes_)
print(train_descriptions.values[0])
print(train_encoded[0])


['Action' 'Adventure' 'Comedy' 'Crime' 'Documentary' 'Horror' 'Romance'
 'Science Fiction' 'Thriller']
A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.
[0 0 1 0 0 0 1 0 0]


In [12]:
%%time

# Need to download Magnitude object before execute. 
# http://magnitude.plasticity.ai/word2vec/medium/GoogleNews-vectors-negative300.magnitude

word2vec = Magnitude('./models/GoogleNews-vectors-negative300.magnitude')

CPU times: user 14.6 ms, sys: 7.46 ms, total: 22 ms
Wall time: 16.7 ms


In [7]:
class BatchSequence(Sequence):
    
    def __init__(self, x_set, y_set, batch_size, shuffle=True):
        data_size = len(x_set)

        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            self.x = x_set[shuffle_indices]
            self.y = y_set[shuffle_indices]
        else:
            self.x, self.y = x_set, y_set
            
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        start_index = idx * self.batch_size
        end_index = min((idx + 1) * self.batch_size, len(self.x))
        
        batch_x = self.x[start_index: end_index]
        batch_y = self.y[start_index: end_index]

        splited_data = list(map(lambda x: x.split(' '), batch_x))
        query_vecs = word2vec.query(splited_data)
        embeddings = np.mean(query_vecs, axis=1)
        
        X, y = np.array(embeddings), np.array(batch_y)

        return X, y


In [8]:
# word2vec embedding dimension
wor2vec_dim = 300

# Input Layers
word_input = Input(shape=(wor2vec_dim, ), dtype=tf.float32)  # (batch_size, sent_length)

# Hidden Layers
x = Dense(64, activation='relu')(word_input)

# Output Layer
predict = Dense(units=9, activation='sigmoid')(x)

model = Model(inputs=[word_input], outputs=predict)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()



Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                19264     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 585       
Total params: 19,849
Trainable params: 19,849
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Create an instance of batch sequence
batch_size = 32

train_batchSequence = BatchSequence(train_descriptions.values, train_encoded, batch_size)
valid_batchSequence = BatchSequence(test_descriptions.values, test_encoded, batch_size)


In [10]:
# Train model

logfile_path = './log'
tb_cb = TensorBoard(log_dir=logfile_path, histogram_freq=0)

history = model.fit_generator(train_batchSequence,
                              epochs=5, 
                              validation_data=valid_batchSequence,
                              callbacks=[tb_cb])

Instructions for updating:
Use tf.cast instead.


Epoch 1/5


  1/375 [..............................] - ETA: 8:27:26 - loss: 0.6935 - acc: 0.4826

  2/375 [..............................] - ETA: 6:02:36 - loss: 0.6929 - acc: 0.5764

  3/375 [..............................] - ETA: 5:21:43 - loss: 0.6923 - acc: 0.6319

...

321/375 [========================>.....] - ETA: 0s - loss: 0.3663 - acc: 0.8453

333/375 [=========================>....] - ETA: 0s - loss: 0.3669 - acc: 0.8448

344/375 [==========================>...] - ETA: 0s - loss: 0.3665 - acc: 0.8450

359/375 [===========================>..] - ETA: 0s - loss: 0.3662 - acc: 0.8453

375/375 [==============================] - 1s 4ms/step - loss: 0.3662 - acc: 0.8453 - val_loss: 0.3239 - val_acc: 0.8699


In [12]:
# Evaluate model

score = model.evaluate_generator(valid_batchSequence)
print('Test score:', score[0])
print('Test accuracy;', score[1])


Test score: 0.3239195086968744
Test accuracy; 0.8699196027930318
